# 지연 계산(Lazy Evaluation)
참조: https://ml-explore.github.io/mlx/build/html/usage/lazy_evaluation.html

### 지연 계산인 이유
MLX를 이용하여 연산을 할 때, 실제로는 아무런 계산이 이루어지지 않습니다. 대신 **계산 그래프(Compute Graph)** 에 기록됩니다.

### 계산 그래프 조작하기
지연 계산은 실제 계산 없이 그저 그래프에 기록을 합니다. 이는 `grad()`와 `vmap()` 같은 함수 변환 및 그래프 최적화에 유용합니다.

현재 MLX에서는 계산 그래프를 컴파일하여 다시 실행하지 않습니다. 계산 그래프는 모두 동적으로 생성됩니다. 그러나 지연 계산은 향후 성능 개선을 위해 **컴파일 통합**을 훨씬 쉽게 만듭니다.

### 사용할 것만 계산하기
MLX에서는 많은 계산양으로 인해 두려워할 필요 없습니다. 다음의 예시를 보시죠.

In [10]:
import mlx.core as mx 
a = mx.array([1,2,3])
b = mx.array([4,5,6])

def add(x,y):
    return x + y
def times(x,y):
    return x * y

def fun1(x,y):
    p = add(x,y)
    q = times(x,y)
    return p, q

result, _ = fun1(a,b)
result

array([5, 7, 9], dtype=int32)

여기서 우리는 실제로 `times`의 출력을 계산하지 않습니다. 그러나 이 패턴을 사용할 때는 주의해야 합니다. `times`의 그래프는 여전히 생성됩니다.

비슷하게, 지연 계산(Lazy evaluation)는 메모리를 절약하면서도 코드를 단순하게 유지하는 데 유리할 수 있습니다.아래의 예시와 같이, `mlx.nn.Module`에서 파생된 매우 큰 모델 `Model`이 있다고 가정해봅시다. 이 모델을 `model = Model()`로 인스턴스화할 수 있습니다. 일반적으로 이는 모든 가중치를 `float32`로 초기화하지만, 실제로 `eval()`을 수행하기 전까지는 아무 계산도 이루어지지 않습니다. 모델을 `float16` 가중치로 업데이트하면, eager computation(즉시 계산)을 사용했을 때보다 소비되는 최대 메모리 양이 절반으로 줄어듭니다.

이 패턴은 MLX에서 지연 계산 덕분에 간단하게 수행할 수 있습니다.
```python
model = Model() # no memory used yet
model.load_weights("weights_fp16.safetensors")
```

### 계산 시점
그렇다면 언제 계산을 할까요? 일반적으로는 `eval()`을 사용할 때라 할 수 있습니다. 여기서 중요한 점은 그래프가 너무 커지는 것과 충분한 유용한 작업을 일괄 처리하지 않는 것 사이의 균형을 맞추는 것입니다.

worst 케이스를 한번 보고 지나가겠습니다. 

In [11]:
def fun2(x,y):
    p = add(x,y)
    mx.eval(p)
    q = times(x,y)
    mx.eval(q)
    return p, q

위 경우는, 각 그래프 계산에는 고정된 오버헤드가 있기 때문입니다. 반면, 계산 그래프의 크기가 커짐에 따라 오버헤드가 약간 증가하기 때문에, 매우 큰 그래프는(계산적으로는 올바르지만) 비용이 많이 들 수 있습니다.

다행히도 MLX에서는 다양한 크기의 계산 그래프가 잘 작동합니다. 계산 당 수십 개에서 수천 개의 연산을 처리하는 정도면 괜찮습니다.

대부분의 수치 계산은 반복적인 외부 루프를 가지고 있습니다(예: 확률적 경사 하강법에서의 반복). eval()을 사용하는 자연스럽고 효율적인 위치는 이 외부 루프의 각 반복에서입니다.

구체적인 예는 다음과 같습니다: (아직은 개념을 학습 중이니, 실제 코드 실행은 나중으로 미룹시다!)
```python
for batch in dataset:
    loss, grad = value_and_grad_fn(model, batch) # 이 단계에서는 아무것도 계산되지 않습니다 .
    optimizer.update(model, grad) # 이 단계에서도 아무것도 계산되지 않습니다 .
    mx.eval(loss, model.parameters()) # 이 단계에서는 모든 것이 계산됩니다 . 
```
중요한 점은 그래프가 암묵적으로 평가되는 시점을 인지하는 것입니다. 배열을 출력하거나, numpy.ndarray로 변환하거나, memoryview를 통해 메모리에 접근할 때마다 그래프가 계산됩니다. save()(또는 다른 MLX 저장 함수)를 통해 배열을 저장하는 것도 배열을 평가합니다.

스칼라 배열에서 array.item()을 호출해도 배열이 계산됩니다. 위의 예에서 loss를 출력하거나(print(loss)), 손실 스칼라 값을 리스트에 추가하는 것(losses.append(loss.item()))은 그래프 계산을 유도합니다. 만약 이러한 코드가 mx.eval(loss, model.parameters()) 이전에 있다면, 이는 부분 계산이 되며, 순방향만 계산됩니다.

또한, 배열이나 배열 집합에 대해 eval()을 여러 번 호출하는 것은 전혀 문제되지 않습니다. 


### 주의

우리가 조심해야할 곳은 아래 함수와 같은 상황입니다. 조건문의 사용으로 인해 계산이 유도될 수 있음에 주의하여야합니다!

In [ ]:
def fun3(x,y):
    p = add(x,y)
    q = times(x,y)
    if p == q: 
        return p
    else:
        return q